In [ ]:
!pip install -q opendatasets

import opendatasets as od
import pandas as pd

In [ ]:
 # insert ypu kaggle  username and key
od.download('https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small',data_dir="dataset")

In [ ]:
df = pd.read_csv('/content/dataset/fashion-product-images-small/styles.csv', on_bad_lines='skip')
df.head()

In [ ]:
sample_df = df.sample(n=2000, random_state=0)
sample_df.head()

In [ ]:
sample_df.describe()

In [ ]:
df.describe()

In [ ]:
!pip install sweetviz
import sweetviz as sv

In [ ]:
sample_report = sv.analyze(sample_df, pairwise_analysis="on")

In [ ]:
sample_report.show_html(layout='vertical')